{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "ab7770b8",
   "metadata": {},
   "source": [
    "# Incremental Update of The Mean and Covariance of Running Samples\n",
    "\n",
    "## Problem Statement\n",
    "\n",
    "We have samples of $\\mathbf{X}=\\{\\mathbf{x}_i\\}_{i=1}^M$ with $M$ degree of freedom (DoF) with $M$ being a finite positive integer. For a sample in a given time, the state vector is $X_n = [x_{1,n}, x_{2,n}, ..., x_{M,n}]$ with $n$ being the sequence number of the sample. \n",
    "\n",
    "The mean of $N$ such samples is $\\overline{X}(N)$ (an M-DoF vector):\n",
    "\n",
    "$$\n",
    "\\overline{X}(N) = \\frac{1}{N} \\Sigma_{n=1}^{N} X_n\n",
    "$$\n",
    "\n",
    "or, to write it in another way:\n",
    "\n",
    "$$\n",
    "\\overline{X}(N) = [\\bar{x}_1(N), \\bar{x}_2(N), ..., \\bar{x}_M(N)] = [\\frac{1}{N}\\Sigma_{n=1}^N x_{1,n}, \\frac{1}{N}\\Sigma_{n=1}^N x_{2,n}, ..., \\frac{1}{N}\\Sigma_{n=1}^N x_{M,n}]\n",
    "$$\n",
    "\n",
    "And the corresponding unbiased covariance $\\sigma_{i,j}(N)$ is:\n",
    "\n",
    "$$\n",
    "\\sigma_{i,j}(N) = \\frac{1}{N-1} \\Sigma_{n=1}^{N} [x_{i,n} - \\bar{x}_i(N)] [x_{j,n} - \\bar{x}_j(N)]\n",
    "$$\n",
    "\n",
    "For a long running sampling process in which $N$ can grow infinitely, we may not be able to keep all samples of $\\mathbf{X}$ to calculate the current mean and covariance. But we can calculate them incrementally with O(1) space complexity.\n",
    "\n",
    "\n",
    "## Derivation\n",
    "\n",
    "Here we derive the formulae to calculate $\\overline{X}(N+1)$ and $\\sigma_{i,j}(N+1)$ given the current sample $X_{N+1}$ and previous mean $\\overline{X}(N)$ and covariance $\\sigma_{i,j}(N)$\n",
    "\n",
    "The new mean is pretty straightforward:\n",
    "\n",
    "$$\n",
    "\\begin{align*}\n",
    "\\overline{X}(N+1) &= \\frac{1}{N+1} \\Sigma_{n=1}^{N+1} X_n \\\\\n",
    "&= \\frac{1}{N+1} (\\Sigma_{n=1}^{N} X_n + X_{N+1}) \\\\\n",
    "&= \\frac{1}{N+1} (N\\overline{X}(N) + X_{N+1})\n",
    "\\end{align*}\n",
    "$$\n",
    "\n",
    "\n",
    "The new covariance is:\n",
    "\n",
    "$$\n",
    "\\begin{align*}\n",
    "\\sigma_{i,j}(N+1) &= \\frac{1}{N} \\Sigma_{n=1}^{N+1} [x_{i,n} - \\bar{x}_i(N+1)] [x_{j,n} - \\bar{x}_j(N+1)] \\\\\n",
    "&= \\frac{1}{N} (\\Sigma_{n=1}^{N} [x_{i,n} - \\bar{x}_i(N+1)] [x_{j,n} - \\bar{x}_j(N+1)] + [x_{i,N+1} - \\bar{x}_i(N+1)] [x_{j,N+1} - \\bar{x}_j(N+1)])\n",
    "\\end{align*}\n",
    "$$\n",
    "\n",
    "\n",
    "\n",
    "Since new term near the end of the equation above $[x_{i,N+1} - \\bar{x}_i(N+1)] [x_{j,N+1} - \\bar{x}_j(N+1)]$ can be easily calculate at this point, we will focus on getting the first term in the brackets:\n",
    "\n",
    "\n",
    "Let's define the mean update (which can also be easily calculated at this point):\n",
    "\n",
    "$$\n",
    "\\Delta\\bar{x}_i (N) = \\bar{x}_i (N+1) - \\bar{x}_i (N)\n",
    "$$\n",
    "\n",
    "Then:\n",
    "\n",
    "$$\n",
    "\\begin{align*}\n",
    "&\\Sigma_{n=1}^{N} [x_{i,n} - \\bar{x}_i(N+1)] [x_{j,n} - \\bar{x}_j(N+1)] \\\\\n",
    "&= \\Sigma_{n=1}^{N} [x_{i,n} - (\\bar{x}_i(N) + \\Delta\\bar{x}_i(N))] [x_{j,n} - (\\bar{x}_j(N) + \\Delta\\bar{x}_j(N))] \\\\\n",
    "&= \\Sigma_{n=1}^{N} [x_{i,n} - \\bar{x}_i(N)][x_{j,n} - \\bar{x}_j(N)] + \\Sigma_{n=1}^{N} [x_{i,n} - \\bar{x}_i (N)] \\Delta\\bar{x}_j(N) + \\Sigma_{n=1}^{N} [x_{j,n} - \\bar{x}_j (N)] \\Delta\\bar{x}_i(N)  + \\Sigma_{n=1}^{N} \\Delta\\bar{x}_i(N) \\Delta\\bar{x}_j(N) \\\\\n",
    "&= \\Sigma_{n=1}^{N} [x_{i,n} - \\bar{x}_i(N)][x_{j,n} - \\bar{x}_j(N)] + \\Sigma_{n=1}^{N} \\Delta\\bar{x}_i(N) \\Delta\\bar{x}_j(N)  \\\\\n",
    "&= (N-1) \\sigma_{i,j}(N) + N \\Delta\\bar{x}_i(N) \\Delta\\bar{x}_j(N)\n",
    "\\end{align*}\n",
    "$$\n",
    "\n",
    "\n",
    "Therefore, the new covariance is:\n",
    "\n",
    "$$\n",
    "\\sigma_{i,j}(N+1) = \\frac{N-1}{N} \\sigma_{i,j}(N) + [\\bar{x}_i(N+1) - \\bar{x}_i(N)] [(\\bar{x}_j(N+1) - \\bar{x}_j(N)] + \\frac{1}{N} [x_{i,N+1} - \\bar{x}_i(N+1)] [x_{j,N+1} - \\bar{x}_j(N+1)]\n",
    "$$\n",
    "\n",
    "\n",
    "## Example Code"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "aa653bb8",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "np_mean: [0.49263978 0.4983813  0.50171179 0.49758825 0.49862414 0.49854833\n",
      " 0.50209534 0.50408652 0.49486627]\n",
      "np_cov:\n",
      "[[ 8.47182198e-02  6.70486401e-04  2.34839021e-04 -1.49486481e-03\n",
      "   1.76914667e-04  7.15548746e-04  3.49897398e-04  2.28240254e-04\n",
      "  -8.87970718e-04]\n",
      " [ 6.70486401e-04  8.22198306e-02 -2.02858671e-04  3.23969968e-04\n",
      "  -1.36810583e-03  1.01783075e-03  8.24766713e-04  3.03025339e-04\n",
      "  -6.15513653e-04]\n",
      " [ 2.34839021e-04 -2.02858671e-04  8.34315486e-02 -3.96289016e-04\n",
      "   1.24640315e-03 -1.14439229e-04  6.95774527e-04 -5.14504935e-05\n",
      "   6.61015953e-04]\n",
      " [-1.49486481e-03  3.23969968e-04 -3.96289016e-04  8.32135561e-02\n",
      "   8.04660923e-04 -8.17886412e-04  7.44674522e-04  9.17769318e-04\n",
      "  -1.84914370e-03]\n",
      " [ 1.76914667e-04 -1.36810583e-03  1.24640315e-03  8.04660923e-04\n",
      "   8.26705046e-02 -1.29864126e-03  1.53125393e-04 -1.24475335e-03\n",
      "  -7.49379362e-04]\n",
      " [ 7.15548746e-04  1.01783075e-03 -1.14439229e-04 -8.17886412e-04\n",
      "  -1.29864126e-03  8.36152597e-02 -6.24623760e-05 -2.19754919e-04\n",
      "  -1.58149452e-04]\n",
      " [ 3.49897398e-04  8.24766713e-04  6.95774527e-04  7.44674522e-04\n",
      "   1.53125393e-04 -6.24623760e-05  8.28039174e-02  4.72624105e-04\n",
      "  -7.97127379e-04]\n",
      " [ 2.28240254e-04  3.03025339e-04 -5.14504935e-05  9.17769318e-04\n",
      "  -1.24475335e-03 -2.19754919e-04  4.72624105e-04  8.33331276e-02\n",
      "  -4.70894847e-05]\n",
      " [-8.87970718e-04 -6.15513653e-04  6.61015953e-04 -1.84914370e-03\n",
      "  -7.49379362e-04 -1.58149452e-04 -7.97127379e-04 -4.70894847e-05\n",
      "   8.46880070e-02]]\n",
      "i_mean: [0.49263978 0.4983813  0.50171179 0.49758825 0.49862414 0.49854833\n",
      " 0.50209534 0.50408652 0.49486627]\n",
      "i_cov:\n",
      "[[ 8.47182198e-02  6.70486401e-04  2.34839021e-04 -1.49486481e-03\n",
      "   1.76914667e-04  7.15548746e-04  3.49897398e-04  2.28240254e-04\n",
      "  -8.87970718e-04]\n",
      " [ 6.70486401e-04  8.22198306e-02 -2.02858671e-04  3.23969968e-04\n",
      "  -1.36810583e-03  1.01783075e-03  8.24766713e-04  3.03025339e-04\n",
      "  -6.15513653e-04]\n",
      " [ 2.34839021e-04 -2.02858671e-04  8.34315486e-02 -3.96289016e-04\n",
      "   1.24640315e-03 -1.14439229e-04  6.95774527e-04 -5.14504935e-05\n",
      "   6.61015953e-04]\n",
      " [-1.49486481e-03  3.23969968e-04 -3.96289016e-04  8.32135561e-02\n",
      "   8.04660923e-04 -8.17886412e-04  7.44674522e-04  9.17769318e-04\n",
      "  -1.84914370e-03]\n",
      " [ 1.76914667e-04 -1.36810583e-03  1.24640315e-03  8.04660923e-04\n",
      "   8.26705046e-02 -1.29864126e-03  1.53125393e-04 -1.24475335e-03\n",
      "  -7.49379362e-04]\n",
      " [ 7.15548746e-04  1.01783075e-03 -1.14439229e-04 -8.17886412e-04\n",
      "  -1.29864126e-03  8.36152597e-02 -6.24623760e-05 -2.19754919e-04\n",
      "  -1.58149452e-04]\n",
      " [ 3.49897398e-04  8.24766713e-04  6.95774527e-04  7.44674522e-04\n",
      "   1.53125393e-04 -6.24623760e-05  8.28039174e-02  4.72624105e-04\n",
      "  -7.97127379e-04]\n",
      " [ 2.28240254e-04  3.03025339e-04 -5.14504935e-05  9.17769318e-04\n",
      "  -1.24475335e-03 -2.19754919e-04  4.72624105e-04  8.33331276e-02\n",
      "  -4.70894847e-05]\n",
      " [-8.87970718e-04 -6.15513653e-04  6.61015953e-04 -1.84914370e-03\n",
      "  -7.49379362e-04 -1.58149452e-04 -7.97127379e-04 -4.70894847e-05\n",
      "   8.46880070e-02]]\n"
     ]
    }
   ],
   "source": [
    "import numpy as np\n",
    "\n",
    "# Generate random samples\n",
    "N_DoF = 9    # degree of freedom of states\n",
    "N_sample = 10000\n",
    "samples = np.random.default_rng().random((N_DoF, N_sample))\n",
    "\n",
    "# Method1: use numpy built-in functions\n",
    "np_mean = np.sum(samples, axis=1) / samples.shape[1]\n",
    "np_cov = np.cov(samples)\n",
    "print(f'np_mean: {np_mean}')\n",
    "print(f'np_cov:\\n{np_cov}')\n",
    "\n",
    "# Method 2: use incremental update above\n",
    "curr_mean = np.zeros(N_DoF)\n",
    "curr_cov = np.zeros((N_DoF,N_DoF))\n",
    "curr_N = 0    # number of samples till the time of calculation\n",
    "for sample in samples.transpose():\n",
    "    if curr_N == 0:\n",
    "        curr_mean += sample\n",
    "    else:\n",
    "        next_mean = (curr_N * curr_mean + sample) / (curr_N + 1)\n",
    "        delta_mean = next_mean - curr_mean\n",
    "        for i in range(9):\n",
    "            for j in range(9):\n",
    "                curr_cov[i][j] = \\\n",
    "                        (curr_N - 1)/curr_N * curr_cov[i][j]    \\\n",
    "                        + delta_mean[i] * delta_mean[j] \\\n",
    "                        + 1/ curr_N * (sample[i] - next_mean[i]) * (sample[j] - next_mean[j])\n",
    "        curr_mean = next_mean\n",
    "    curr_N += 1\n",
    "print(f'i_mean: {curr_mean}')\n",
    "print(f'i_cov:\\n{curr_cov}')"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
